# xcube resampling test - rectification of an OLCI scene to EPSG:4326

In [ ]:
import glob
import xarray as xr
import numpy as np
import dask.array as da
import pyproj
import matplotlib.pyplot as plt
from dask.distributed import Client
from xcube_resampling.grid import Grid
from xcube_resampling.rectifier import Rectifier

In [ ]:
%%time
path = '/windows/tmp/eopf/S3A_OL_1_EFR____20210801T102426_20210801T102726_20210802T141313_0179_074_336_2160_LN1_O_NT_002.SEN3'
l1b = xr.open_mfdataset([path + '/geo_coordinates.nc'] +
                        [(path + '/Oa{:02d}_radiance.nc'.format(x)) for x in range(1, 22)],
                       chunks=2048)

In [ ]:
l1b

In [ ]:
%%time
#dst_grid = Grid(pyproj.CRS("EPSG:4326"), (0, 0), (0.003, -0.003), (0, 0), (2400, 2400))
dst_grid = Grid(pyproj.CRS("EPSG:4326"), (-10.863, 52.449), (0.003, -0.003), (6667, 4304), (2400, 2400))
r = Rectifier(l1b['longitude'].data, l1b['latitude'].data, dst_grid)
r.prepare_forward_index()
r.prepare_src_bboxes()
r.compute_forward_index()
#r.determine_covering_dst_grid()
print(r.dst_grid)

In [ ]:
%time
r.prepare_inverse_index()

In [ ]:
%%time
r.prepare_rectification(l1b['Oa12_radiance'].data, l1b['Oa08_radiance'].data)

In [ ]:
r.dask_rectified.visualize()

In [ ]:
%%time
result = r.compute_rectification()
result

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(5, 5))
plt.imshow(result[1], cmap='RdYlGn');

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(4, 4))
plt.imshow(l1b['Oa12_radiance'], cmap='RdYlGn');